# Exploring the CIA World Factbook

In the DataQuest mission _**Guided Project: Working With a SQLite Database**_, there are three next steps to explore using the CIA World  Factbook (https://www.cia.gov/library/publications/the-world-factbook/):
    * Which countries will lose population over the next 35 years?
    * Which countries have the lowest and highest population densities?
    * Which countries receive the most immigrants? Which countries lose the most emigrants?
    
The data was originally loaded from "The World Factbook": https://github.com/factbook/factbook.sql/releases

However, I noticed that population_growth and migration_rate did not have any negative values. I downloaded the correct data from the CIA website (population growth(https://www.cia.gov/library/publications/the-world-factbook/fields/2002.html#ic) and migration rates (https://www.cia.gov/library/publications/the-world-factbook/fields/2112.html#ic) and reset these fields accordingly. The updated database is here in factbook.db.


In [1]:
#Import the needed modules
import sqlite3
import math
import pandas as pd

In [2]:
#Define a function for population growth
def predict_growth(row, years):
    n = row["population"]*(math.e**((row["population_growth"]/100)*years))
    return int(round(n,0))

In [3]:
# Connect to the database one time for all steps
conn = sqlite3.connect("factbook.db")

## Which countries will lose population over the next 35 years?


The formula for compound annual population growth is:
![growth rate](http://bit.ly/2xtnMfH)
Where ![N](http://bit.ly/2xtq8ex) is the final population; ![N.sub-zero](http://bit.ly/2fgJR7f) is the initial population, _e_  is the math constant, r is the growth rate and t is the number of years we want to project out


- Read in the name of the country, the current population and the population growth
  - Filter out null data 
  - Filter out the World and the European Union
  - Order by growth
- Use pandas' SQL Reader to run the query
- Use numpy's apply (built into pandas) to run the formula for each row and insert in a new column
- Print the 5 countries with the highest population, and the 5 countries with the lowest population

In [4]:
condition_list = ["name", "population", "population_growth"]
num_of_conds = len(condition_list)

query = "select name, population, population_growth from facts where ("
for idx in range(num_of_conds):
    if idx == 0:
        query += condition_list[idx]+" is not null "
    else:
        query += " and "+condition_list[idx]+" is not null "
        
query +=  ") and code != 'ee' and code != 'xx' "
query += " order by population desc;"
growth_df=pd.read_sql_query(query,conn)

growth_df["new_population"] = growth_df.apply(predict_growth, axis= 1, args=(35,))
last_rows = len(growth_df) - 5

print ("The countries with the highest and lowest populations are:")
print (growth_df.iloc[0:5], '\n')
print (growth_df.iloc[last_rows:])

The countries with the highest and lowest populations are:
            name  population  population_growth  new_population
0          China  1367485388              0.004      1369401208
1          India  1251695584              0.012      1256963761
2  United States   321368864              0.008       322269958
3      Indonesia   255993674              0.009       256801325
4         Brazil   204259812              0.007       204760862 

                        name  population  population_growth  new_population
229                  Tokelau        1337               0.01            1342
230                     Niue        1190               0.03            1203
231  Holy See (Vatican City)         842               0.00             842
232  Cocos (Keeling) Islands         596               0.00             596
233         Pitcairn Islands          48               0.00              48


## Which countries have the lowest and highest population densities?


- Read in the name of the country, the current population and the land area
  - Filter out null and zero data 
  - Filter out the World and the European Union
  - Order by density
- Use pandas' SQL Reader to run the query
- Use pandas to calculate density
- Print the 5 countries with the highest densities, and the 5 countries with the lowest densities

In [5]:
condition_list = ["area_land", "population"]
num_of_conds = len(condition_list)

query = "select name, population, area_land from facts where ("
for idx in range(num_of_conds):
    if idx == 0:
        query = query + "("+condition_list[idx]+" is not null and "+ condition_list[idx]+" > 0)"
    else:
        query = query + " and ("+condition_list[idx]+" is not null and " + condition_list[idx]+" > 0)"
        
query +=  ") and code != 'ee' and code != 'xx' "
query += " order by (population/area_land) desc;"
pop_area = pd.read_sql_query(query,conn) 
pop_area["pop_density"] = pop_area["population"]/pop_area["area_land"]
last_rows = len(pop_area)- 5

print ("The countries with the highest and lowest densities are:")
print (pop_area.iloc[0:5], '\n')
print (pop_area.iloc[last_rows:])

The countries with the highest and lowest densities are:
         name  population  area_land   pop_density
0       Macau      592731         28  21168.964286
1      Monaco       30535          2  15267.500000
2   Singapore     5674472        687   8259.784571
3   Hong Kong     7141106       1073   6655.271202
4  Gaza Strip     1869055        360   5191.819444 

                                  name  population  area_land  pop_density
227                           Mongolia     2992908    1553556     1.926489
228                   Pitcairn Islands          48         47     1.021277
229                          Greenland       57733    2166086     0.026653
230                           Svalbard        1872      62045     0.030172
231  Falkland Islands (Islas Malvinas)        3361      12173     0.276103


## Which countries receive the most immigrants? Which countries lose the most emigrants?


- Read in the name of the country and the migration rate
  - Filter out null data 
  - Filter out the World and the European Union
  - Order by migration rate
- Use pandas' SQL Reader to run the query
- Print the 5 countries with the highest migration rate, and the 5 countries with the lowest migration rate

In [6]:
condition_list = ["migration_rate"]
num_of_conds = len(condition_list)

query = "select name, migration_rate from facts where ( migration_rate is not null"
query +=  ") and code != 'ee' and code != 'xx' "
query += " order by migration_rate desc;"
mig_rates = pd.read_sql_query(query,conn) 
last_rows = len(mig_rates)-5

print ("Which countries receive the most immigrants?")
print (mig_rates.iloc[0:5], '\n')
print("Which countries lose the most emigrants?:")
print (mig_rates.iloc[last_rows:])

Which countries receive the most immigrants?
                     name  migration_rate
0                   Syria            61.5
1  British Virgin Islands            16.5
2              Luxembourg            15.5
3                   Qatar            14.6
4          Cayman Islands            13.8 

Which countries lose the most emigrants?:
                                name  migration_rate
218                      Puerto Rico           -16.9
219                            Tonga           -17.8
220                          Lebanon           -20.3
221  Micronesia, Federated States of           -20.9
222                   American Samoa           -26.7


In [ ]:
#Close the connection -- we're done
conn.close()

## Final notes:

I was surprised too.  A couple of things:

- This is the rate per 1,000 people, not the total number
- the rate was -113.51 migrant(s)/1,000 population in 2014 (the last year of data before 2017)
- the rate was 61.5 migrant(s)/1,000 population in 2017.  
      That's a huge jump especially in regards to the immigration crisis we hear so much about.

I think Syria absorbed (and continues to absorb) refugees from Iraq, Iran and Lebanon.  This apparently more than compensates for the refugees leaving for Europe.

The data may be wrong, but I don't know of any other source.  I might try and find an XML reader  so I can look at previous years more easily.